#### Métricas de las semillas

Al estar las muestras equilibradas para el entramiento de los modelos, la exactitud (accuracy), la precisión (precision), la sensibilidad (recall) y la puntuación F1 adquieren más relevancia y se vuelven más confiables para comparar el rendimiento de los modelos. 

El equilibrio de las clases reduce el sesgo hacia la clase mayoritaria, que es una de las principales limitaciones de estas métricas en conjuntos de datos desequilibrados.


In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import psutil
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from commons import myfunctions as myfunc

start_time = datetime.datetime.now()


Leer fichero con métricas de los modelos entrenados con varias semillas

In [ ]:
archivo1= os.path.join(myfunc.MET_DIR, "metricas_semillas.csv")

fichero1 = os.path.basename(archivo1)
carpeta1 = os.path.dirname(archivo1)

df_tmp = myfunc.read_csv_to_df_spa(fichero1, carpeta1)[['tipo', 'select', 'clasific', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'roc_auc_ovr','fichero_modelo','indices_auc','indices_jaccard']]
df_tmp['tipo_select_clasific'] = df_tmp['tipo'] + '-' + df_tmp['select'] + '-' + df_tmp['clasific']
df_tmp['select_clasific'] = df_tmp['select'] + '-' + df_tmp['clasific']
df_tmp['auc'] = df_tmp['roc_auc'].fillna(0) + df_tmp['roc_auc_ovr'].fillna(0)

df_c = df_tmp.copy()
df1 = df_c.sort_values(["tipo","select","clasific"])

Comprobar la gráfica con la métrica de la exactitud

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
sns.set_style("whitegrid")

media_accuracy = df1.groupby(['tipo', 'select_clasific'])['accuracy'].mean().reset_index()

for i, tipo in enumerate(df1['tipo'].unique()):
    ax = axes[i // 2, i % 2]
    current_palette = sns.color_palette("muted", n_colors=len(df1['select_clasific'].unique()))
    sns.boxplot(x='select_clasific', y='accuracy', data=df1[df1.tipo == tipo], ax=ax, palette=current_palette)
    ax.set_title(f'Tipo: {tipo}-{myfunc.EXAMPLES_MAPPING[tipo]}')

    # Encontrar el 'select_clasific' con la mejor media de auc para este 'tipo'
    mejor_media = media_accuracy[media_accuracy['tipo'] == tipo]['accuracy'].max()

    # Comprobar que hay una mejor media antes de continuar
    if not media_accuracy[(media_accuracy['tipo'] == tipo) & (media_accuracy['accuracy'] == mejor_media)].empty:
        mejor_select_clasific = media_accuracy[(media_accuracy['tipo'] == tipo) & (media_accuracy['accuracy'] == mejor_media)]['select_clasific'].values[0]

        # Obtener la posición x de la mejor media
        x_posicion_mejor_media = df1[df1['tipo'] == tipo]['select_clasific'].unique().tolist().index(mejor_select_clasific)

        # Añadir la línea vertical punteada en la posición de la mejor media
        ax.axvline(x=x_posicion_mejor_media, color='red', linestyle='--', lw=1)

        # Añadir la anotación de la mejor media
        ax.annotate(f'Mejor: {mejor_media:.4f}', xy=(x_posicion_mejor_media, mejor_media),
                    xytext=(50, 30), textcoords='offset points',
                    arrowprops=dict(facecolor='black', shrink=0.1), ha='center')
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.show()


#### Número de veces que se ha entrenado cada modelo.



- por cada 4 tipo de conjunto de datos:

    -  se han entrenado para los tres algoritmos de selección (ANOVA, MI, RF) 
    
        - y por cada uno de esos 3, se han utilizado los 5 de clasificación(LR-EN, LR-L1, LR-L2, RN, SVM)

En total 60 por cada semilla.

In [ ]:
df1 = df_c.sort_values(["tipo","select","clasific"])
df_conta = df1.groupby("tipo_select_clasific").size()

myfunc.verbose(f"Modelos distintos por los 4 conjunto de datos: {len(df_conta)}")

myfunc.verbose("Numero de veces que se ha entrenado cada modelo:")
display(df_conta)

#### Las medias y desviaciones de cada modelo por cada métrica.

In [ ]:
df1 = df_c.sort_values(["tipo","select","clasific"])

columnas = ["accuracy", "precision", "recall", "f1_score", "auc"]

df_media = df1.groupby("tipo_select_clasific")[columnas].mean()
df_std = df1.groupby("tipo_select_clasific")[columnas].std()
df_suma = df_media + df_std

df_merge = df_media.merge(df_std, left_index=True, right_index=True)
df_merge = df_merge.merge(df_suma, left_index=True, right_index=True)


Métricas para los modelos de clasificación binaria con muestras reales

In [ ]:
myfunc.verbose("Media de los modelos con el conjunto de datos bin_s, solo muestras reales")

display(len(df_merge))
display(df_merge[df_merge.index.str.startswith("bin_s")])

Métricas para los modelos de clasificación binaria con muestras reales y sintéticas

In [ ]:
myfunc.verbose("Media de los modelos con el conjunto de datos bin_m, muestras reales y sintéticas")

display(len(df_merge))
display(df_merge[df_merge.index.str.startswith("bin_m")])

Métricas para los modelos de clasificación multicáncer con muestras reales

In [ ]:
myfunc.verbose("Media de los modelos con el conjunto de datos mul_s, solo muestras reales")

display(len(df_merge))
display(df_merge[df_merge.index.str.startswith("mul_s")])

Métricas para los modelos de clasificación multicáncer con muestras reales y sintéticas

In [ ]:
myfunc.verbose("Media de los modelos con el conjunto de datos mul_s, muestras reales y sintéticas")

display(len(df_merge))
display(df_merge[df_merge.index.str.startswith("mul_m")])

#### Columnas que interesan y funciones para mínimos para las desviaciones y máximos para el resto.

In [ ]:
columns_media = ["accuracy_x", "precision_x", "recall_x", "f1_score_x", "auc_x"] 
columns_std = ["accuracy_y", "precision_y", "recall_y", "f1_score_y", "auc_y"]
columns_suma = ["accuracy", "precision", "recall", "f1_score", "auc"]


def get_max_by_column(df):
  max_by_column = {}
  for column in df.columns:
    max_by_column[column] = df[column].idxmax(), df[column].max()

  return max_by_column


def get_min_by_column(df):
  min_by_column = {}
  for column in df.columns:
    min_by_column[column] = df[column].idxmin(), df[column].min()

  return min_by_column


def mostrar_mejores(tipo1, estadistico1, df1, orden_columnas=["accuracy"], ordenacion1=False):
  df_tipo = df1.query(f"tipo_select_clasific.str.startswith('{tipo1}')")
  if ordenacion1 == False:
    # búsqueda de la mayor media
    valores1 = pd.DataFrame(get_max_by_column(df_tipo))
  else:
    # búsqueda de la menor desviación
    valores1 = pd.DataFrame(get_min_by_column(df_tipo))

  algoritmos = valores1.iloc[0].unique()
  columnas = df_tipo.columns
  df_tipo = df_tipo.round(4)
  
  myfunc.verbose(f"Algoritmos en primera posicion por alguna de sus columnas para conjunto de datos {tipo1} y estadístico {estadistico1}")

  display(df_tipo[df_tipo.index.isin(algoritmos)].sort_values(orden_columnas, ascending=ordenacion1))


#### Resumen de los mejores por accuracy

In [ ]:
orden_columnas = ["accuracy_x"]
tipo1 = 'bin_s'
mostrar_mejores(tipo1, "media", df_merge[columns_media], orden_columnas)
tipo1 = 'bin_m'
mostrar_mejores(tipo1, "media", df_merge[columns_media], orden_columnas)
tipo1 = 'mul_s'
mostrar_mejores(tipo1, "media", df_merge[columns_media], orden_columnas)
tipo1 = 'mul_m'
mostrar_mejores(tipo1, "media", df_merge[columns_media], orden_columnas)


#### Finalización del notebook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")